<a href="https://colab.research.google.com/github/EltonGS/covid-19_previsao_leito_UTI/blob/main/Notebooks/1_tratamento_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução (sobre minha pessoa)

Olá a todos!

Seja bem vindo a esse projeto de Ciência de Dados. Contando um pouco sobre mim e o que me levou a conhecer e me interessar por essa área:

Em meados de março de 2021 a empresa onde trabalho disponibilizou uma série de cursos para que os funcionários estivessem cientes das tecnologias e temas que são extremamente importantes para o crescimentos dos clientes. Um desses cursos era sobre Dados e como eles são utilizados para impulsionar o crescimento de empresas, as tomadas de decisões, e como a utilização de dados vem aumentando e ainda há muito a ser explorado.

A maneira como os dados podem ser utilizados para extrair informações valiosas sobre os mais diferentes temas me chamou a atenção e a partir dali comecei a pesquisar mais sobre isso, buscando como é essa área de atuação e o que é necessário para fazer parte dela.

Por um golpe do destino, em abril de 2021 a Alura abriu inscrição para a 3ª Imersão de Dados. Um curso intensivo de 1 semana mostrando os principais pontos de um projeto de Ciência de Dados. Ali eu não tinha conhecimento algum sobre a linguagem de programação Python e suas bibliotecas utilizadas ou sobre os modelos matemáticos e estatísticos mencionados durante as aulas, mas a maneira como os dados foram manipulados e analisados me interessaram muito.

Logo após a imersão foi aberta inscrição para o Bootcamp de Data Sciente Aplicada. Uma experiência de 12 semanas aprendendo sobre Python, as bibliotecas utilizadas para manipulação e visualização dos dados, análises de séries temporais e modelos de Machine Learning.

Esse é o projeto final desse Bootcamp de Data Science Aplicada que foi disponilizado pela [Alura Cursos Online](https://www.alura.com.br/). Nele serão aplicados os conhecimentos adquiridos nas últimas 12 semanas:
- Tratamento dos dados;
- Análise Exploratória e visualização dos dados;
- Criação de Modelo de Machine Learning;

Espero que esse projeto possa te auxiliar de alguma maneira, ou pelo menos te despertar o interesse em conhecer mais sobre essa área.

# Introdução (do projeto)

Em dezembro de 2019 foi relatado o primeiro caso oficial de **Covid-19** causado pelo vírus SARS-CoV-2. No Brasil, em 26 de fevereiro de 2020 foi registrado o primeiro caso de Covid-19. Essa nova doença causou um estresse enorme sobre os sistemas de saúde dos países por não existir um tratamento com eficácia cientificamente comprovada. Para um parcela das pessoas infectadas os sintomas evoluem e se tornam severos ou críticos e assim necessitam de maiores cuidados. Esses pacientes precisam ser transferidos para a UTI e receber oxigênio.

Com o movimento desses pacientes para a UTI, ocorreu um **colapso do sistema de saúde** de diversos países ao redor do mundo, incluindo o **Brasil**.

Medidas foram tomadas para **aumentar o número de leitos de UTI** nos hospitais brasileiros, e também hospitais de campanha foram criados para atender um número maior de pacientes.

Para utilizar os leitos de UTI da melhor maneira possível, o Hospital Sírio-Libanês disponibilizou uma base de dados no Kaggle (pode ser verificada [aqui](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)) para incentivar a criação de Modelo de Machine Learning (Aprendizado de Máquina) que possam prever se um paciente necessitará ou não de um leito de UTI com base nos exames médicos realizados quando o paciente está no hospital.

Dessa maneira os pacientes que necessitam de um leito de UTI podem ser acomodados rapidamente em um leito livre, ou que possa ser buscado um leito em outro local, e os pacientes que não necessitarão de um leito podem ser acompanhados remotamente.

**Esse projeto explorará a base de dados fornecida pelo Hospital Sírio-Libanês e criará modelos de Machine Learning que façam a previsão da necessidade, ou não, de um leito de UTI.**

# Informações do Projeto

Esse projeto está dividido em 3 notebooks, conforme a descrição a seguir:

* **Tratamento dos dados**: onde os dados serão tratados e manipulados.

* **Análise Exploratória e Visualização dos Dados**: onde os dados serão analisados e visualizados buscando o entendimento da base de dados.

* **Modelo de Machine Learning**: onde serão criados os modelos de Machine Learning para previsão da necessidade de um leito de UTI para os pacientes com Covid-19.



**Bibliotecas Utilizadas**

As bibliotecas utilizadas nesse projeto são importadas na célula abaixo.





In [1]:
# bibliotecas para manipulação e tratamento dos dados
import pandas as pd
import numpy as np

# bibliotecas para visualização dos dados
import matplotlib.pyplot as plt
import seaborn as sns

# módulo para pré-processamento dos dados
from sklearn import preprocessing

# Funções

As funções abaixo foram criadas para tornar o manuseio dos dados mais fácil de ser compreendido.

In [2]:
def dimensao_dados (dados):
  # verifica as dimensões do dataframe
  linha, coluna = dados.shape

  # imprime as dimensões do dataframe
  return print(f'O dataframe possui {linha} linhas e {coluna} colunas.')

In [3]:
def preenche_ausente (dados_originais):
  """
  Essa função preenche valores ausentes com os valores anteriores e posteriores
  """
  # seleciona o nome das colunas com dados contínuos
  colunas_dados_continuos = dados_originais.iloc[:, 13:-2].columns

  # preenche os dados ausentes nas colunas com dados contínuos
  dados_continuos = dados_originais.groupby(by='patient_visit_identifier', as_index=False)[colunas_dados_continuos].fillna(method='ffill').fillna(method='bfill')

  # seleciona as colunas com dados categóricos
  dados_categoricos = dados_originais.iloc[:, :13]

  # seleciona as colunas WINDOW e ICU
  saida = dados_originais.iloc[:, -2:]

  # contatena os dados após o preenchimento dos dados ausentes
  dados_finais = pd.concat([dados_categoricos, dados_continuos, saida], ignore_index = True, axis=1)
  
  dados_finais.columns = dados_originais.columns

  return dados_finais

In [4]:
def prepare_window(rows):
  """
  Essa função identifica se o paciente em algum momento foi para a UTI,
  e preenche essa informação na coluna ICU da janela 0-2.
  Retorna somente a linha da janela 0-2
  """
  if (np.any(rows['icu'])):
    rows.loc[rows['window'] == '0-2', 'icu'] = 1
  return rows.loc[rows['window'] == '0-2']

In [5]:
def remove_corr_var(dados, valor_corte):
  """
  Essa função verificar a correlação entre as features (colunas) do dataframe
  E exclui as colunas do dataframe que será utilizado no modelo de Machine Learning
  """
  # calcula a matriz de correlação
  matriz_corr = dados.iloc[:, 4:-2].corr().abs()

  # separa os valores com base no triângulo superior da matriz e atribue valor bool
  matriz_superior = matriz_corr.where(np.triu(np.ones(matriz_corr.shape), k=1).astype(np.bool))

  # verifica as colunas acima do valor informado como alta correlação
  excluir = [coluna for coluna in matriz_superior.columns if any(matriz_superior[coluna] > valor_corte)]

  return dados.drop(excluir, axis=1)

# Carregando os Dados

In [6]:
# endereço dos dados
url = 'https://github.com/EltonGS/covid_ML_previsao_UTI/blob/main/Dados/Dados%20Originais/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true'

# carrega os dados
dados = pd.read_excel(url)

# visualiza os dados
dados

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,...,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,WINDOW,ICU
0,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4-6,0
3,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,-0.107143,0.736842,NaN,NaN,NaN,NaN,0.318681,0.898990,NaN,NaN,NaN,NaN,-0.275362,0.736842,NaN,NaN,NaN,NaN,-1.000000,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,6-12,0
4,0,1,60th,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,-1.0,-0.871658,-0.871658,-0.871658,-0.871658,-1.0,-0.863874,-0.863874,-0.863874,-0.863874,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.414634,-0.414634,-0.414634,-0.414634,-1.0,-0.979069,-0.979069,...,-0.996762,-1.0,-0.243021,-0.338537,-0.213031,-0.317859,0.033779,0.665932,-0.283951,-0.376923,-0.188679,-0.379310,0.035714,0.631579,-0.340206,-0.4875,-0.572650,-0.857143,0.098901,0.797980,-0.076923,0.286486,0.298507,0.272727,0.362319,0.947368,-0.339130,0.325153,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.4075

## Informações Sobre os Dados

## Informações compartilhadas pelo Sírio Libanês

O **Hospital Sírio Libanês** compartilhou algumas informações sobre os dados:

- A coluna **ICU** é a que deve ser prevista utilizando um modelo de Machine Learning.

- A partir do momento que o valor na coluna ICU é 1 os dados não devem ser utilizados para a o modelo de previsão pois não é possível dizer se os exames foram feitos antes ou depois do paciente ser admitido na UTI.

- Os dados são anonimizados e foram coletados nas unidades de São Paulo e Brasília. Os dados foram anonimizados utilizando as melhores práticas internacionais.

- Os dados foram tratados e colocados em escala utilizando o método **Min Max Scaler** para estarem entre -1 e 1.

- Os dados disponíveis nessa base são:
> * Informação demográfica do paciente (03 colunas);
> * Agrupamento de doenças já existentes no paciente (09 colunas);
> * Resultados de exame de sangue (36 colunas);
> * Sinais vitais (06 colunas);

A equipe do Sírio Libanês também adicionou colunas com valores matemáticos e estatistícos dos exames de sangue e sinais vitais. Estes estão representados nas colunas com o nomes terminando em MEAN (média), MEDIAN (mediana), MAX (valor máximo), MIN (valor mínimo), DIFF (diferença entre o MAX e o MIN) e o DIFF_REL (A razão entre o DIFF e o MEDIAN).

**Dados Ausentes** - Os dados da tabela são provenientes de exames que são realizados com frequências diferentes. Por exemplo, medição dos sinais vitais são feitas a cada hora e exames laboratoriais de sangue são feitos semanalmente.

É razoável assumir que se os dados estão faltando para uma janela de tempo para um paciente é porque o paciente se manteve estável e apresentou resultados similares aos que já estavam registrados nas janelas anteriores ou posteriores. Assim os valores das janelas vizinhas podem ser utilizados para preencher os dados ausentes.

## Análise Primária da Base de Dados

Com os dados importados é possível verificar algumas informações.

Por exemplo, quais as dimensões dessa base de dados? Quantas linhas e colunas?

In [7]:
# visualiza a dimensão da base de dados
dimensao_dados(dados)

O dataframe possui 1925 linhas e 231 colunas.


A primeira coluna nos dados é a **PATIENT_VISIT_IDENTIFIER**, primeiramente parece ser uma linha para cada paciente. Assim teríamos informações sobre 1925 pacientes.

Será que isso é o correto?

In [8]:
# descreve os dados
dados.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEDIAN,ALBUMIN_MEAN,ALBUMIN_MIN,ALBUMIN_MAX,ALBUMIN_DIFF,BE_ARTERIAL_MEDIAN,BE_ARTERIAL_MEAN,BE_ARTERIAL_MIN,BE_ARTERIAL_MAX,BE_ARTERIAL_DIFF,BE_VENOUS_MEDIAN,BE_VENOUS_MEAN,BE_VENOUS_MIN,BE_VENOUS_MAX,BE_VENOUS_DIFF,BIC_ARTERIAL_MEDIAN,BIC_ARTERIAL_MEAN,BIC_ARTERIAL_MIN,BIC_ARTERIAL_MAX,BIC_ARTERIAL_DIFF,BIC_VENOUS_MEDIAN,BIC_VENOUS_MEAN,BIC_VENOUS_MIN,BIC_VENOUS_MAX,BIC_VENOUS_DIFF,BILLIRUBIN_MEDIAN,BILLIRUBIN_MEAN,BILLIRUBIN_MIN,...,DIMER_MIN,DIMER_MAX,DIMER_DIFF,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,BLOODPRESSURE_DIASTOLIC_MEDIAN,BLOODPRESSURE_SISTOLIC_MEDIAN,HEART_RATE_MEDIAN,RESPIRATORY_RATE_MEDIAN,TEMPERATURE_MEDIAN,OXYGEN_SATURATION_MEDIAN,BLOODPRESSURE_DIASTOLIC_MIN,BLOODPRESSURE_SISTOLIC_MIN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,TEMPERATURE_MIN,OXYGEN_SATURATION_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_SISTOLIC_MAX,HEART_RATE_MAX,RESPIRATORY_RATE_MAX,TEMPERATURE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_DIASTOLIC_DIFF,BLOODPRESSURE_SISTOLIC_DIFF,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
count,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,821.000000,821.0,821.000000,821.000000,821.000000,...,821.000000,821.000000,821.0,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1925.000000
mean,192.000000,0.467532,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,0.158333,0.809896,0.528527,0.528527,0.528527,0.528527,-1.0,-0.963433,-0.963433,-0.963433,-0.963433,-1.0,-0.931121,-0.931121,-0.931121,-0.931121,-1.0,-0.310924,-0.310924,-0.310924,-0.310924,-1.0,-0.311845,-0.311845,-0.311845,-0.311845,-1.0,-0.945928,-0.945928,-0.945928,...,-0.954177,-0.954177,-1.0,-0.093631,-0.332600,-0.264701,-0.438754,0.066893,0.743077,-0.097790,-0.338468,-0.268632,-0.435121,0.063798,0.748588,-0.040855,-0.207812,-0.264999,-0.483129,0.326823,0.817565,-0.235001,-0.399582,-0.282029,-0.316753,0.014964,0.818593,-0.752454,-0.728053,-0.754100,-0.703683,-0.770338,-0.887196,-0.786997,-0.715950,-0.817800,-0.719147,-0.771327,-0.886982,0.267532
std,111.168431,0.499074,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,0.365148,0.392485,0.224100,0.224100,0.224100,0.224100,0.0,0.160870,0.160870,0.160870,0.160870,0.0,0.169509,0.169509,0.169509,0.169509,0.0,0.100256,0.100256,0.100256,0.100256,0.0,0.118812,0.118812,0.118812,0.118812,0.0,0.076610,0.076610,0.076610,...,0.123582,0.123582,0.0,0.252064,0.274102,0.246760,0.217113,0.242858,0.132635,0.257733,0.277952,0.252709,0.225554,0.249208,0.125994,0.281304,0.277802,0.272725,0.278239,0.216198,0.283453,0.271123,0.287580,0.296247,0.402675,0.276163,0.141316,0.364001,0.408677,0.366349,0.482097,0.319001,0.296147,0.324754,0.419103,0.270217,0.446600,0.317694,0.296772,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000

Na imagem acima é possível identificar colunas que possuem dados categóricos (são 1 ou 0) como a **AGE_ABOVE65, GENDER e ICU**, e colunas com dados contínuos (variam entre -1 e 1) como **ALBUMIN_MEAN e ALBUMIN_MIN**. As colunas com dados contínuos foram normalizadas pelo Hospital Sírio Libanês.

Com essa verificação é possível extrair outras informações:

- A coluna **PATIENT_VISIT_IDENTIFIER** possui valores que vão de 0 a 384. Com essa informação é possível identificar que cada linha não representa um paciente, são 385 pacientes. Cada paciente tem 5 linhas de informação nos dados.

- A coluna **ICU** tem os valores 0 e 1:
> * **0** - Paciente não está na UTI;
> * **1** - Paciente está na UTI;

- A coluna **GENDER** tem os valores 0 e 1:
> * **0** - Paciente do sexo masculino;
> * **1** - Paciente do sexo feminino;

- A coluna **WINDOW** possui valores para identificar o momento que os exames foram feitos em cada paciente, as janelas para os exames são:
> * **0-2** - Exames feitos entre o momento que o paciente chegou ao hospital e 2 horas após a entrada.
> * **2-4** - Exames feitos entre 2 e 4 horas da entrada do paciente no hospital.
> * **4-6** - Exames feitos entre 4 e 6 horas da entrada do paciente no hospital.
> * **6-12** - Exames feitos entre 6 e 12 horas da entrada do paciente no hospital.
> * **Above-12** - Exames feitos após 12 horas da entrada do paciente no hospital.

A equipe do Hospital Sírio Libanês compartilhou explicação para algumas das colunas, essas informações também foram utilizadas na explicação acima. Essas informações podem ser conferidas no [Kaggle](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19).



# Tratamento dos Dados

Conforme informado pela equipe do Sírio Libanês, existem dados ausentes na base de dados e esses dados podem ser preenchidos utilizando os valores presentes nas janelas anteriores ou posteriores.

Esses valores ausentes estão nas colunas com valores contínuos com as medições dos exames, a equipe do Sírio Libanês informou que essas informações iniciam na 13ª coluna e seguem até a antepenúltima.

Para maior praticidade os nomes das colunas serão alterados para letras minúsculas.

In [9]:
# renomea os nomes das colunas para letras minúsculas
dados.columns = dados.columns.str.lower()
dados.columns

Index(['patient_visit_identifier', 'age_above65', 'age_percentil', 'gender',
       'disease grouping 1', 'disease grouping 2', 'disease grouping 3',
       'disease grouping 4', 'disease grouping 5', 'disease grouping 6',
       ...
       'temperature_diff', 'oxygen_saturation_diff',
       'bloodpressure_diastolic_diff_rel', 'bloodpressure_sistolic_diff_rel',
       'heart_rate_diff_rel', 'respiratory_rate_diff_rel',
       'temperature_diff_rel', 'oxygen_saturation_diff_rel', 'window', 'icu'],
      dtype='object', length=231)

In [10]:
# verifica a quantidade de valores ausentes
dados_ausentes = dados.isna().sum().sum()
print(f'O total de dados ausentes é: {dados_ausentes} valores')

O total de dados ausentes é: 223863 valores


Para realizar o preenchimento dos dados ausentes será usada a função **preenche_ausente**.

In [11]:
dados_limpos = preenche_ausente(dados)

In [12]:
# verifica a quantidade de dados ausentes.
ausente_depois_preenchido = dados_limpos.isna().sum().sum()
print(f'O total de dados ausentes é: {ausente_depois_preenchido} valores')

O total de dados ausentes é: 45 valores


Ainda existem dados ausentes após o preenchimento ser realizado nos dados com valores contínuos.

In [13]:
# descreve os dados
dados_limpos.describe()

,patient_visit_identifier,age_above65,gender,disease grouping 1,disease grouping 2,disease grouping 3,disease grouping 4,disease grouping 5,disease grouping 6,htn,immunocompromised,other,albumin_median,albumin_mean,albumin_min,albumin_max,albumin_diff,be_arterial_median,be_arterial_mean,be_arterial_min,be_arterial_max,be_arterial_diff,be_venous_median,be_venous_mean,be_venous_min,be_venous_max,be_venous_diff,bic_arterial_median,bic_arterial_mean,bic_arterial_min,bic_arterial_max,bic_arterial_diff,bic_venous_median,bic_venous_mean,bic_venous_min,bic_venous_max,bic_venous_diff,billirubin_median,billirubin_mean,billirubin_min,...,dimer_min,dimer_max,dimer_diff,bloodpressure_diastolic_mean,bloodpressure_sistolic_mean,heart_rate_mean,respiratory_rate_mean,temperature_mean,oxygen_saturation_mean,bloodpressure_diastolic_median,bloodpressure_sistolic_median,heart_rate_median,respiratory_rate_median,temperature_median,oxygen_saturation_median,bloodpressure_diastolic_min,bloodpressure_sistolic_min,heart_rate_min,respiratory_rate_min,temperature_min,oxygen_saturation_min,bloodpressure_diastolic_max,bloodpressure_sistolic_max,heart_rate_max,respiratory_rate_max,temperature_max,oxygen_saturation_max,bloodpressure_diastolic_diff,bloodpressure_sistolic_diff,heart_rate_diff,respiratory_rate_diff,temperature_diff,oxygen_saturation_diff,bloodpressure_diastolic_diff_rel,bloodpressure_sistolic_diff_rel,heart_rate_diff_rel,respiratory_rate_diff_rel,temperature_diff_rel,oxygen_saturation_diff_rel,icu
count,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,...,1925.000000,1925.000000,1925.0,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000,1925.000000
mean,192.000000,0.467532,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,0.158333,0.809896,0.550349,0.550349,0.550349,0.550349,-1.0,-0.981793,-0.981793,-0.981793,-0.981793,-1.0,-0.946031,-0.946031,-0.946031,-0.946031,-1.0,-0.314653,-0.314653,-0.314653,-0.314653,-1.0,-0.318061,-0.318061,-0.318061,-0.318061,-1.0,-0.946194,-0.946194,-0.946194,...,-0.954252,-0.954252,-1.0,-0.072529,-0.328239,-0.261038,-0.446696,0.072480,0.748537,-0.076075,-0.332671,-0.263690,-0.439785,0.069267,0.752796,0.009399,-0.169805,-0.229606,-0.448757,0.355901,0.844893,-0.258253,-0.434771,-0.318550,-0.382920,-0.021645,0.805126,-0.818498,-0.805300,-0.823067,-0.796211,-0.831911,-0.919693,-0.843716,-0.796734,-0.869248,-0.807560,-0.832776,-0.919584,0.267532
std,111.168431,0.499074,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,0.365148,0.392485,0.187898,0.187898,0.187898,0.187898,0.0,0.109844,0.109844,0.109844,0.109844,0.0,0.141756,0.141756,0.141756,0.141756,0.0,0.067938,0.067938,0.067938,0.067938,0.0,0.105340,0.105340,0.105340,0.105340,0.0,0.059767,0.059767,0.059767,...,0.134259,0.134259,0.0,0.256547,0.275713,0.247852,0.216000,0.248730,0.134322,0.261753,0.278592,0.253902,0.226178,0.255550,0.131652,0.275071,0.269218,0.265246,0.262153,0.210648,0.233901,0.246646,0.265763,0.273801,0.353062,0.265408,0.142677,0.316782,0.354669,0.321999,0.406496,0.287052,0.243907,0.282808,0.364214,0.235885,0.375520,0.285875,0.244444,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

Na descrição dos dados acima é visto que existem 1925 valores para a coluna **patient_visit_identifier** mas para as colunas com os dados sobre as doenças já existentes no paciente há 1920 valores. Como cada paciente possui 5 linhas com informações, um paciente não possui essas informações preenchidas nos dados. Como não há como saber quais as informações desse paciente, essas linhas podem ser excluídas.

In [14]:
# exclui as linhas com informação ausente
dados_limpos = dados_limpos.dropna()

In [15]:
# verifica a quantidade de dados ausentes.
ausente_depois_preenchido = dados_limpos.isna().sum().sum()
print(f'O total de dados ausentes é: {ausente_depois_preenchido}')

O total de dados ausentes é: 0


In [16]:
# descreve os dados
dados_limpos.describe()

,patient_visit_identifier,age_above65,gender,disease grouping 1,disease grouping 2,disease grouping 3,disease grouping 4,disease grouping 5,disease grouping 6,htn,immunocompromised,other,albumin_median,albumin_mean,albumin_min,albumin_max,albumin_diff,be_arterial_median,be_arterial_mean,be_arterial_min,be_arterial_max,be_arterial_diff,be_venous_median,be_venous_mean,be_venous_min,be_venous_max,be_venous_diff,bic_arterial_median,bic_arterial_mean,bic_arterial_min,bic_arterial_max,bic_arterial_diff,bic_venous_median,bic_venous_mean,bic_venous_min,bic_venous_max,bic_venous_diff,billirubin_median,billirubin_mean,billirubin_min,...,dimer_min,dimer_max,dimer_diff,bloodpressure_diastolic_mean,bloodpressure_sistolic_mean,heart_rate_mean,respiratory_rate_mean,temperature_mean,oxygen_saturation_mean,bloodpressure_diastolic_median,bloodpressure_sistolic_median,heart_rate_median,respiratory_rate_median,temperature_median,oxygen_saturation_median,bloodpressure_diastolic_min,bloodpressure_sistolic_min,heart_rate_min,respiratory_rate_min,temperature_min,oxygen_saturation_min,bloodpressure_diastolic_max,bloodpressure_sistolic_max,heart_rate_max,respiratory_rate_max,temperature_max,oxygen_saturation_max,bloodpressure_diastolic_diff,bloodpressure_sistolic_diff,heart_rate_diff,respiratory_rate_diff,temperature_diff,oxygen_saturation_diff,bloodpressure_diastolic_diff_rel,bloodpressure_sistolic_diff_rel,heart_rate_diff_rel,respiratory_rate_diff_rel,temperature_diff_rel,oxygen_saturation_diff_rel,icu
count,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.0,1920.000000,1920.000000,1920.000000,1920.000000,1920.0,1920.000000,1920.000000,1920.000000,1920.000000,1920.0,1920.000000,1920.000000,1920.000000,1920.000000,1920.0,1920.000000,1920.000000,1920.000000,1920.000000,1920.0,1920.000000,1920.000000,1920.000000,...,1920.000000,1920.000000,1920.0,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000
mean,191.981771,0.468750,0.369792,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,0.158333,0.809896,0.550206,0.550206,0.550206,0.550206,-1.0,-0.981746,-0.981746,-0.981746,-0.981746,-1.0,-0.945891,-0.945891,-0.945891,-0.945891,-1.0,-0.314647,-0.314647,-0.314647,-0.314647,-1.0,-0.318064,-0.318064,-0.318064,-0.318064,-1.0,-0.946213,-0.946213,-0.946213,...,-0.954190,-0.954190,-1.0,-0.073586,-0.328894,-0.260981,-0.446491,0.072576,0.748705,-0.077141,-0.333337,-0.263640,-0.439583,0.069355,0.752974,0.008269,-0.170573,-0.229781,-0.448810,0.355769,0.844804,-0.258725,-0.434989,-0.318252,-0.382418,-0.021286,0.805441,-0.818025,-0.804793,-0.822607,-0.795680,-0.831473,-0.919483,-0.843309,-0.796205,-0.868908,-0.807059,-0.832341,-0.919374,0.268229
std,111.312587,0.499152,0.482874,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,0.365148,0.392485,0.188122,0.188122,0.188122,0.188122,0.0,0.109984,0.109984,0.109984,0.109984,0.0,0.141914,0.141914,0.141914,0.141914,0.0,0.068027,0.068027,0.068027,0.068027,0.0,0.105477,0.105477,0.105477,0.105477,0.0,0.059843,0.059843,0.059843,...,0.134428,0.134428,0.0,0.256042,0.275773,0.248172,0.216244,0.249047,0.134457,0.261257,0.278649,0.254230,0.226438,0.255877,0.131777,0.274535,0.269147,0.265569,0.262493,0.210906,0.234200,0.246793,0.266075,0.274095,0.353385,0.265660,0.142729,0.317059,0.354992,0.322291,0.406892,0.287297,0.244190,0.283063,0.364540,0.236097,0.375880,0.286119,0.244728,0.443153
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

Agora não existem dados ausentes na base de dados. E todas as colunas tem uma contagem de 1920 valores.

Esses serão os dados utilizados para a **Análise Exploratória dos Dados (EDA)**.

In [17]:
# exporta os dados em um arquivo csv
dados_limpos.to_csv('dados_eda.csv', index=False)

## Tratamento dos dados para Machine Learning

Ainda são necessárias alterações nos dados para que possam ser utilizados para o modelo de Machine Learning.

É possível verificar quais os tipos dos dados que existem na base de dados.

In [18]:
# informações sobre os dados
dados_limpos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1920 entries, 0 to 1924
Columns: 231 entries, patient_visit_identifier to icu
dtypes: float64(225), int64(4), object(2)
memory usage: 3.4+ MB


Na informação acima é identificado que a base de dados possui:
- 225 colunas com dados do tipo **float64**;
- 4 colunas com dados do tipo **int64**;
- 2 colunas com dados do tipo **object**;

As colunas com dados do tipo **int64** são referente aos valores categóricos e os dados do tipo **float64** são dos valores contínuos.

As colunas com os dados do tipo **object** são as **age_percentil** e **window**. Pois possuem dados do tipo **string**.

A coluna **age_percentil** pode ser transformada em dados categóricos, assim se tornando um dado numérico. Para isso será utilizada a função **LabelEncoder()** presente no módulo PreProcessing da biblioteca **SciKit Learn**.

In [19]:
# visualiza os valores únicas da coluna age_percentil
dados_limpos['age_percentil'].unique()

array(['60th', '90th', '10th', '40th', '70th', '20th', '50th', '80th',
       '30th', 'Above 90th'], dtype=object)

In [20]:
# define o LabelEncoder na variável le
le = preprocessing.LabelEncoder()

# transforma os dados string em dados categóricos
dados_limpos['age_percentil'] = le.fit_transform(dados_limpos['age_percentil'])

# visualiza os valores da coluna age_percentil após a transformação
dados_limpos['age_percentil'].unique()

array([5, 8, 0, 3, 6, 1, 4, 7, 2, 9])

Após a transformação essa coluna também poderá ser utilizada na criação do modelo de Machine Learning, a coluna window continuará sendo do tipo objeto.

Como informado pelo Sírio Libanês, a partir do momento que o paciente foi para a UTI os dados não devem ser utilizado para o modelo. E também o melhor é identificar a necessidade de um leito de UTI para o paciente o mais rapidamente possível, para isso será selecionada somente a primeira janela (**0-2**) para o modelo de Machine Learning desse projeto.

Primeiramente serão identificados os dados onde o paciente entrou diretamente para a UTI logo na primeira janela.

In [21]:
# identifica os dados de pacientes que chegaram ao hospital e foram para a UTI
a_remover = dados_limpos.query("window == '0-2' and icu==1")['patient_visit_identifier'].values

# remove os paciente que foram diretamente para a UTI
dados_limpos = dados_limpos.query("patient_visit_identifier not in @a_remover")

# visualiza os dados após a remoção
dados_limpos

,patient_visit_identifier,age_above65,age_percentil,gender,disease grouping 1,disease grouping 2,disease grouping 3,disease grouping 4,disease grouping 5,disease grouping 6,htn,immunocompromised,other,albumin_median,albumin_mean,albumin_min,albumin_max,albumin_diff,be_arterial_median,be_arterial_mean,be_arterial_min,be_arterial_max,be_arterial_diff,be_venous_median,be_venous_mean,be_venous_min,be_venous_max,be_venous_diff,bic_arterial_median,bic_arterial_mean,bic_arterial_min,bic_arterial_max,bic_arterial_diff,bic_venous_median,bic_venous_mean,bic_venous_min,bic_venous_max,bic_venous_diff,billirubin_median,billirubin_mean,...,dimer_max,dimer_diff,bloodpressure_diastolic_mean,bloodpressure_sistolic_mean,heart_rate_mean,respiratory_rate_mean,temperature_mean,oxygen_saturation_mean,bloodpressure_diastolic_median,bloodpressure_sistolic_median,heart_rate_median,respiratory_rate_median,temperature_median,oxygen_saturation_median,bloodpressure_diastolic_min,bloodpressure_sistolic_min,heart_rate_min,respiratory_rate_min,temperature_min,oxygen_saturation_min,bloodpressure_diastolic_max,bloodpressure_sistolic_max,heart_rate_max,respiratory_rate_max,temperature_max,oxygen_saturation_max,bloodpressure_diastolic_diff,bloodpressure_sistolic_diff,heart_rate_diff,respiratory_rate_diff,temperature_diff,oxygen_saturation_diff,bloodpressure_diastolic_diff_rel,bloodpressure_sistolic_diff_rel,heart_rate_diff_rel,respiratory_rate_diff_rel,temperature_diff_rel,oxygen_saturation_diff_rel,window,icu
0,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
1,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,2-4,0
2,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,0.333333,-0.230769,-0.132075,-0.586207,0.535714,0.578947,0.443299,0.0000,-0.025641,-0.500000,0.714286,0.838384,-0.076923,-0.459459,-0.313433,-0.636364,0.246377,0.578947,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,4-6,0
3,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.333333,-0.230769,-0.132075,-0.593220,-0.107143,0.736842,0.333333,-0.230769,-0.132075,-0.586207,-0.107143,0.736842,0.443299,0.0000,-0.025641,-0.500000,0.318681,0.898990,-0.076923,-0.459459,-0.313433,-0.636364,-0.2753

Após a remoção dos paciente que foram diretamente para a UTI temos 1760 linhas na base de dados.

A função **prepare_window** será usada para identificar se um paciente foi para a UTI em algum momento e selecionar somente a primeira janela nos dados.

In [22]:
# aplica a função após agrupar os dados pelo identificador de paciente
dados_limpos = dados_limpos.groupby(by='patient_visit_identifier').apply(prepare_window)

# visualiza os dados
dados_limpos.head()

,,patient_visit_identifier,age_above65,age_percentil,gender,disease grouping 1,disease grouping 2,disease grouping 3,disease grouping 4,disease grouping 5,disease grouping 6,htn,immunocompromised,other,albumin_median,albumin_mean,albumin_min,albumin_max,albumin_diff,be_arterial_median,be_arterial_mean,be_arterial_min,be_arterial_max,be_arterial_diff,be_venous_median,be_venous_mean,be_venous_min,be_venous_max,be_venous_diff,bic_arterial_median,bic_arterial_mean,bic_arterial_min,bic_arterial_max,bic_arterial_diff,bic_venous_median,bic_venous_mean,bic_venous_min,bic_venous_max,bic_venous_diff,billirubin_median,billirubin_mean,...,dimer_max,dimer_diff,bloodpressure_diastolic_mean,bloodpressure_sistolic_mean,heart_rate_mean,respiratory_rate_mean,temperature_mean,oxygen_saturation_mean,bloodpressure_diastolic_median,bloodpressure_sistolic_median,heart_rate_median,respiratory_rate_median,temperature_median,oxygen_saturation_median,bloodpressure_diastolic_min,bloodpressure_sistolic_min,heart_rate_min,respiratory_rate_min,temperature_min,oxygen_saturation_min,bloodpressure_diastolic_max,bloodpressure_sistolic_max,heart_rate_max,respiratory_rate_max,temperature_max,oxygen_saturation_max,bloodpressure_diastolic_diff,bloodpressure_sistolic_diff,heart_rate_diff,respiratory_rate_diff,temperature_diff,oxygen_saturation_diff,bloodpressure_diastolic_diff_rel,bloodpressure_sistolic_diff_rel,heart_rate_diff_rel,respiratory_rate_diff_rel,temperature_diff_rel,oxygen_saturation_diff_rel,window,icu
patient_visit_identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,0.086420,-0.230769,-0.283019,-0.586207,-0.285714,0.736842,0.237113,0.0000,-0.162393,-0.500000,0.208791,0.898990,-0.247863,-0.459459,-0.432836,-0.636364,-0.420290,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
2,10,2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.938950,-0.938950,...,-0.978029,-1.0,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.506173,-0.815385,-0.056604,-0.517241,0.357143,0.947368,-0.525773,-0.5125,-0.111111,-0.714286,0.604396,0.959596,-0.435897,-0.491892,0.000000,-0.575758,0.101449,1.000000,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,-0.515528,-0.351328,-0.747001,-0.756272,-1.000000,-0.961262,0-2,1
3,15,3,0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-0.263158,-0.263158,-0.263158,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.972789,-0.972789,...,-0.978029,-1.0,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,0.012346,-0.369231,-0.528302,-0.448276,-0.285714,0.684211,0.175258,-0.1125,-0.384615,-0.357143,0.208791,0.878788,-0.299145,-0.556757,-0.626866,-0.515152,-0.420290,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
4,20,4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,0.605263,0.605263,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.317073,-0.317073,-0.317073,-0.317073,-1.0,-0.935113,-0.935113,...,-1.000000,-1.0,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,0.333333,-0.153846,0.160377,-0.586207,0.285714,0.868421,0.443299,0.0000,0.196581,-0.571429,0.538462,0.939394,-0.076923,-0.351351,-0.044776,-0.575758,0.072464,0.894737,-1.000000,-0.877301,-0.9

In [23]:
# verifica as dimensões do dataframe
dimensao_dados(dados_limpos)

O dataframe possui 352 linhas e 231 colunas.


Após a verificação se os pacientes foram para a UTI, ou não, e a seleção somente da primeira janela na coluna window temos um dataframe com **352 linhas**, agora com **cada linha sendo um paciente**.

Outros conceitos importante ao trabalhar com um modelo de Machine Learning são **underfitting** e **overfitting**. Esses conceitos podem ser definidos como:

- **underfitting**: o modelo não consegue aprender o suficiente com os dados. Erros altos tanto nos dados de treino como nos dados de teste.

- **overfitting**: quando o modelo aprende demais com os dados de treino e não consegue generalizar. Grande performance com os dados de treino mas falha com os dados de teste.

Uma maneira de evitar o **overfitting** é verificar se existem features (colunas) com **alta correlação nos dados** e remover. Quanto mais próximo de 1 ou -1 o valor maior é a correlação entre as variáveis. Uma maneira de verificar isso é através de uma **matriz de correlação**.

O dataframe possui colunas com dados categóricos e esses não faz sentido verificar correlação.

In [24]:
dados_limpos.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 352 entries, (0, 0) to (384, 1920)
Columns: 231 entries, patient_visit_identifier to icu
dtypes: float64(225), int64(5), object(1)
memory usage: 652.2+ KB


É necessário verificar a **correlação** entre os dados do tipo **float64**. As colunas com os valores dos exames realizados nos pacientes.

In [25]:
# seleciona todos os tipos de dados exceto os float64
dados_limpos.select_dtypes(exclude='float64')

,,patient_visit_identifier,age_above65,age_percentil,gender,window,icu
patient_visit_identifier,,,,,,,
0,0,0,1,5,0,0-2,1
2,10,2,0,0,0,0-2,1
3,15,3,0,3,1,0-2,0
4,20,4,0,0,0,0-2,0
5,25,5,0,0,0,0-2,0
...,...,...,...,...,...,...,...
380,1900,380,0,3,1,0-2,1
381,1905,381,1,9,0,0-2,0
382,1910,382,0,4,0,0-2,1


Acima temos as colunas que possuem dados categóricos ou object. São as 4 primeiras e as 2 últimas colunas da base de dados. Essas colunas podem ser removidas ao chamar a função da matriz de correlação.

In [26]:
# função para calcular a correlação e excluir as features com alta correlação
dados_ml = remove_corr_var(dados_limpos, 0.9)
dados_ml.head()

,,patient_visit_identifier,age_above65,age_percentil,gender,disease grouping 1,disease grouping 2,disease grouping 3,disease grouping 4,disease grouping 5,disease grouping 6,htn,immunocompromised,other,albumin_median,albumin_diff,be_arterial_median,be_arterial_diff,be_venous_median,be_venous_diff,bic_arterial_median,bic_arterial_diff,bic_venous_median,bic_venous_diff,billirubin_median,billirubin_diff,blast_median,blast_diff,calcium_median,calcium_diff,creatinin_median,creatinin_diff,ffa_median,ffa_diff,ggt_median,ggt_diff,glucose_median,glucose_diff,hematocrite_median,hematocrite_diff,hemoglobin_diff,...,pcr_median,pcr_diff,ph_arterial_median,ph_arterial_diff,ph_venous_median,ph_venous_diff,platelets_median,platelets_diff,potassium_median,potassium_diff,sat02_arterial_median,sat02_arterial_diff,sat02_venous_median,sat02_venous_diff,sodium_median,sodium_diff,tgo_median,tgo_diff,tgp_median,tgp_diff,ttpa_median,ttpa_diff,urea_median,urea_diff,dimer_median,dimer_diff,bloodpressure_diastolic_mean,bloodpressure_sistolic_mean,heart_rate_mean,respiratory_rate_mean,temperature_mean,oxygen_saturation_mean,bloodpressure_diastolic_diff,bloodpressure_sistolic_diff,heart_rate_diff,respiratory_rate_diff,temperature_diff,oxygen_saturation_diff,window,icu
patient_visit_identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.938950,-1.0,-1.0,-1.0,0.183673,-1.0,-0.868365,-1.0,-0.742004,-1.0,-0.945093,-1.0,-0.891993,-1.0,0.090147,-1.0,-1.0,...,-0.875236,-1.0,0.234043,-1.0,0.363636,-1.0,-0.540721,-1.0,-0.518519,-1.0,0.939394,-1.0,0.345679,-1.0,-0.028571,-1.0,-0.997201,-1.0,-0.990854,-1.0,-0.825613,-1.0,-0.836145,-1.0,-0.994912,-1.0,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,1
2,10,2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.938950,-1.0,-1.0,-1.0,0.357143,-1.0,-0.912243,-1.0,-0.742004,-1.0,-0.958528,-1.0,-0.780261,-1.0,0.144654,-1.0,-1.0,...,-0.939887,-1.0,0.234043,-1.0,0.363636,-1.0,-0.399199,-1.0,-0.703704,-1.0,0.939394,-1.0,0.345679,-1.0,0.085714,-1.0,-0.995428,-1.0,-0.986662,-1.0,-0.846633,-1.0,-0.836145,-1.0,-0.978029,-1.0,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,-0.547826,-0.533742,-0.603053,-0.764706,-1.000000,-0.959596,0-2,1
3,15,3,0,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.263158,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.972789,-1.0,-1.0,-1.0,0.326531,-1.0,-0.968861,-1.0,-0.194030,-1.0,-0.316589,-1.0,-0.891993,-1.0,-0.203354,-1.0,-1.0,...,-0.503592,-1.0,0.234043,-1.0,0.363636,-1.0,-0.564753,-1.0,-0.777778,-1.0,0.939394,-1.0,0.580247,-1.0,0.200000,-1.0,-0.989549,-1.0,-0.956555,-1.0,-0.846633,-1.0,-0.937349,-1.0,-0.978029,-1.0,0.012346,-0.369231,-0.528302,-0.457627,-0.285714,0.684211,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0-2,0
4,20,4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.935113,-1.0,-1.0,-1.0,0.357143,-1.0,-0.913659,-1.0,-0.829424,-1.0,-0.938084,-1.0,-0.851024,-1.0,0.358491,-1.0,-1.0,...,-0.990926,-1.0,0.234043,-1.0,0.363636,-1.0,-0.457944,-1.0,-0.592593,-1.0,0.939394,-1.0,0.345679,-1.0,0.142857,-1.0,-0.998507,-1.0,-0.991235,-1.0,-0.846633,-1.0,-0.903614,-1.0,-1.000000,-1.0,0.333333,-0.153846,0.160377,-0.593220,0.285714,0.868421,-1.000000,-0.877301,-0.923664,-0.882353,-0.952381,-0.979798,0-2,0
5,25,5,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.0,-1.0,-1.0,-1.0,-1.0,-0.317073,-1.0,-0.317073,-1.0,-0.938950,-1.0,-1.0,-1.0,0.357143,-1.0,-0.891012,-1.0,-0.742004,-1.0,-0.958528,-1.0,-0.891993,-1.0,0.291405,-1.0,-1.0,...,-0.997732,-1.0,0.234043,-1.0,0.363636,-1.0,-0.292390,-1.0,-0.666667,-1.0,0.939394,-1.0,0.345679,-1.0,0.085714,-1.0,-0.997947,-1.0,-0.988948,-1.0,-0.846633,-1.0,-0.884337,-1.0,-1.000000,-1.0,-0.037037,-0.

In [27]:
# verifica o tamando do dataset após a remoção das features com alta correlação
dimensao_dados(dados_ml)

O dataframe possui 352 linhas e 97 colunas.


Após a remoção dos dados com alta correlação o dataframe está com 97 colunas.

Esses são os tratamentos que será realizados aos dados nesse notebook.

In [28]:
# exporta a base de dados em um arquivo csv
dados_ml.to_csv('dados_ml.csv', index=False)

# Conclusão

Ao final desse notebook 2 bases de dados são exportadas.

A base de dados dados_limpos será utilizada para a análise exploratória e visualização dos dados no notebook analise_dos_dados.

A base de dados dados_ml será utilizada para a criação dos modelos de machine learning no notebook modelos_machine_learning.

## Referências utilizadas durante o projeto

1. [Página do Kaggle com informações sobre a base de dados](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)

2. [Text classification with extremely small datasets](https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2)

3. [5 Feature Selection Method From SciKit Learn You Should Know](https://towardsdatascience.com/5-feature-selection-method-from-scikit-learn-you-should-know-ed4d116e4172)

4. [Como Selecionar As Melhores Features Para Seu Modelo de Machine Learning](https://paulovasconcellos.com.br/como-selecionar-as-melhores-features-para-seu-modelo-de-machine-learning-2e9df83d062a)

5. [Machine Learning Map](https://scikit-learn.org/stable/tutorial/machine_learning_map/)

6. [Primeiro caso de Covid-19 pode ter atingindo a Chine em outubro](https://www.cnnbrasil.com.br/saude/primeiro-caso-covid-19-pode-ter-atingido-a-china-em-outubro-de-2019-diz-estudo/)

7. [Evolução dos Sintomas do Covid-19](https://saude.abril.com.br/medicina/casos-sem-sintomas-evolucoes-coronavirus/)

8. [Covid Impact to Global Healthcare](https://edition.cnn.com/videos/world/2021/04/14/covid-impact-global-healthcare-lon-orig.cnn)

9. [Observatório Covid-19 aponta maior colapso sanitário e hospitalar da história do Brasil](https://portal.fiocruz.br/noticia/observatorio-covid-19-aponta-maior-colapso-sanitario-e-hospitalar-da-historia-do-brasil)

10. [Primeiro caso Covid-19 no Brasil](https://saude.abril.com.br/medicina/coronavirus-primeiro-caso-brasil/)

11. [Overfitting e Underfitting em Machine Learning](https://abracd.org/overfitting-e-underfitting-em-machine-learning/#:~:text=Underfitting%20%C3%A9%20uma%20tradu%C3%A7%C3%A3o%20para,Overfitting%20%C3%A9%20o%20oposto)

12. [WHO delivers advice and support for older people during Covid-19](https://www.who.int/news-room/feature-stories/detail/who-delivers-advice-and-support-for-older-people-during-covid-19#:~:text=The%20COVID%2D19%20pandemic,potential%20underlying%20health%20conditions)

13. [Machine Learning in Healthcare](https://healthinformatics.uic.edu/blog/machine-learning-in-healthcare/)

14. [Machine Learning](https://www.ibm.com/cloud/learn/machine-learning)

15. [Hiperparameter Tuning](https://towardsdatascience.com/hyperparameter-tuning-c5619e7e6624)

16. [SKLearn Model Selection - RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

17. [Entenda o que é AUC e ROC nos modelos de Machine Learning](https://medium.com/bio-data-blog/entenda-o-que-%C3%A9-auc-e-roc-nos-modelos-de-machine-learning-8191fb4df772)

18. [Understanding Confusion Matrix](https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62)

19. [Understanding a Classification Report For Your Machine Learning Model](https://medium.com/@kohlishivam5522/understanding-a-classification-report-for-your-machine-learning-model-88815e2ce397)

20. [Understanding AUC ROC Curve](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5)

21. [Regressão Logística](https://pt.wikipedia.org/wiki/Regress%C3%A3o_log%C3%ADstica)

22. [SciKit Learn - RandomForestClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

23. [SciKit Learn - Logistic Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

24. [SciKit Learn - SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

25. [SciKit Learn - LinearSVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)